In [2]:
import gurobipy as gp
from gurobipy import GRB
from pathlib import Path
import os
import numpy as np
import sys




In [3]:
INSTANCE_PATH = Path('../instances/sifaleras')
RESULT_PATH   = Path('../RESULTADOS/')

In [4]:
arq = open(os.path.join(INSTANCE_PATH,'52_1.txt'))
N = int(arq.readline())



In [5]:
instances_names = os.listdir(INSTANCE_PATH)

In [6]:
def leitura_instance(file_name):
    arq = open(os.path.join(INSTANCE_PATH,file_name))
    N = int(arq.readline())
    PP  = [0]*N
    PR  = [0]*N

    FP = [float(arq.readline())]*N
    FR = [float(arq.readline())]*N

    HR = [float(arq.readline())]*N
    HP = [float(arq.readline())]*N

    D = [int(i) for i in arq.readline().split()]
    
    R = [int(i) for i in arq.readline().split()]
    
    return N, PP, PR, FP, FR, HR, HP, D, R

In [7]:
cap = True
file_name = '52_1.txt' #sys.argv[1]
N, PP, PR, FP, FR, HR, HP, D, R =leitura_instance(file_name)
MAX_CPU_TIME = 600.0
EPSILON = 0.000001


In [8]:
SD = (np.zeros((N,N))).tolist()
SR = (np.zeros((N,N))).tolist()
for  i in range(N):
    SD[i][i] = D[i]
    SR[i][i] = R[i]
    for j in range(i+1, N):
        SD[i][j] = SD[i][j-1] + D[j]
        SR[i][j] = SR[i][j-1] + R[j]
    
soma = sum(D)
fator = 1.5
#Capacidade de cada período
C = (soma * fator)/N

try:

    # Create a new model
    model = gp.Model("CLSR")

    # Create variables
    
    xp = model.addVars(list(range(N)), lb =0.0, ub = float('inf'),vtype=GRB.CONTINUOUS, name="xp")
    yp = model.addVars(list(range(N)), lb =0.0, ub = 1.0,vtype=GRB.BINARY, name="yp")
    sp = model.addVars(list(range(N)), lb =0.0, ub = float('inf'),vtype=GRB.CONTINUOUS, name="sp")
    xr = model.addVars(list(range(N)), lb =0.0, ub = float('inf'),vtype=GRB.CONTINUOUS, name="xr")
    yr = model.addVars(list(range(N)), lb =0.0, ub = 1.0,vtype=GRB.BINARY, name="yr")
    sr = model.addVars(list(range(N)), lb =0.0, ub = float('inf'),vtype=GRB.CONTINUOUS, name="sr")


    # # Set objective
    model.setObjective(gp.quicksum(PP[i]*xp[i]+sp[i]*HP[i] + xr[i]*PR[i] + sr[i]*HR[i] + yp[i]*FP[i] + yr[i]*FR[i] for i in range(N)) , sense = GRB.MINIMIZE)

    # # Add constraints
    
    model.addConstr(xp[0]+xr[0]-sp[0] == D[0])
    model.addConstrs(sp[i-1] + xp[i] + xr[i] - sp[i] == D[i] for i in range(N) if i > 0 )
    model.addConstr(R[0] - xr[0] - sr[0] == 0)
    model.addConstrs(sr[i-1] + R[i] - xr[i] - sr[i] == 0 for i in range(N) if i > 0)
    model.addConstrs(xp[i] - yp[i]*min(C,SD[i][N-1]) <= 0 for i in range(N))
    model.addConstrs(xr[i] - yr[i]*min(SR[0][i], SD[i][N-1]) <= 0 for i in range(N))
    model.addConstrs(xp[i] + xr[i] <= C for i in range(N))
    model.write(file_name+".lp")

    # Parameters 
    model.setParam(GRB.Param.TimeLimit, MAX_CPU_TIME)
    model.setParam(GRB.Param.MIPGap, EPSILON)
    model.setParam(GRB.Param.Threads,1)
    model.setParam(GRB.Param.Cuts, -1)
    model.setParam(GRB.Param.Presolve,-1)




    # model.set(gp.GRB_DoubleParam_MIPGap, EPSILON)
    # model.set(gp.GRB_IntParam_Threads, 1)
    # model.set(gp.GRB_IntParam_Cuts, -1)
    # model.set(gp.GRB_IntParam_Presolve, -1)



    # Optimize model
    model.optimize()




    print('Obj: %g' % model.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')


Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-25
Set parameter TimeLimit to value 60
Set parameter MIPGap to value 1e-06
Set parameter Threads to value 1
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads
Optimize a model with 260 rows, 312 columns and 674 nonzeros
Model fingerprint: 0xba15b155
Variable types: 208 continuous, 104 integer (104 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+02]
Found heuristic solution: objective 12770.800000
Presolve removed 7 rows and 7 columns
Presolve time: 0.00s
Presolved: 253 rows, 305 columns, 660 nonzeros
Variable types: 203 continuous, 102 integer (102 binary)

Root relaxation: objective 7.197681e+03, 229 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds     

In [23]:
a = [1,2,3,4,5]

In [24]:
sum(a)

15